In [9]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os

# Load model
model = load_model("./violence_detection_model.h5")
class_names = ['Normal', 'Violence', 'Weapon']
input_shape = model.input_shape[1:]  # Should be (15, 128, 128, 3)

def create_frame_sequence(video_path, seq_length=15):
    """Extracts sequences of frames from video"""
    cap = cv2.VideoCapture(video_path)
    frames = []
    sequences = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Preprocess each frame
        frame = cv2.resize(frame, (input_shape[2], input_shape[1]))  # (128, 128)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = frame / 255.0  # Normalize
        
        frames.append(frame)
        
        # When we have enough frames for a sequence
        if len(frames) == seq_length:
            sequences.append(np.array(frames))
            frames = frames[1:]  # Sliding window (optional)
    
    cap.release()
    return np.array(sequences)

def classify_video(video_path):
    """Classifies video using 3D model"""
    # Create sequences of 15 frames each
    sequences = create_frame_sequence(video_path)
    
    if len(sequences) == 0:
        print("Not enough frames in video")
        return
    
    # Process each sequence
    for i, seq in enumerate(sequences):
        # Add batch dimension (1, 15, 128, 128, 3)
        seq = np.expand_dims(seq, axis=0)
        
        # Predict
        predictions = model.predict(seq, verbose=0)
        pred_class = class_names[np.argmax(predictions[0])]
        confidence = np.max(predictions[0])
        
        print(f"Sequence {i+1}: {pred_class} ({confidence:.2f})")

# Example usage
video_path = "./t_n001_converted.avi"
classify_video(video_path)

Sequence 1: Normal (0.00)
Sequence 2: Normal (0.00)
Sequence 3: Normal (0.00)
Sequence 4: Normal (0.00)
Sequence 5: Normal (0.00)
Sequence 6: Normal (0.00)
Sequence 7: Normal (0.00)
Sequence 8: Normal (0.00)
Sequence 9: Normal (0.00)
Sequence 10: Normal (0.00)
Sequence 11: Normal (0.00)
Sequence 12: Normal (0.00)
Sequence 13: Normal (0.00)
Sequence 14: Normal (0.00)
Sequence 15: Normal (0.00)
Sequence 16: Normal (0.00)
Sequence 17: Normal (0.00)
Sequence 18: Normal (0.00)
Sequence 19: Normal (0.00)
Sequence 20: Normal (0.00)
Sequence 21: Normal (0.00)
Sequence 22: Normal (0.00)
Sequence 23: Normal (0.00)
Sequence 24: Normal (0.00)
Sequence 25: Normal (0.00)
Sequence 26: Normal (0.00)
Sequence 27: Normal (0.00)
Sequence 28: Normal (0.00)
Sequence 29: Normal (0.00)
Sequence 30: Normal (0.00)
Sequence 31: Normal (0.00)
Sequence 32: Normal (0.00)
Sequence 33: Normal (0.00)
Sequence 34: Normal (0.00)
Sequence 35: Normal (0.00)
Sequence 36: Normal (0.00)
Sequence 37: Normal (0.00)
Sequence 3

In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os
from collections import deque

# Load model and verify output shape
model = load_model("./violence_detection_model.h5")
print("Model input shape:", model.input_shape)
print("Model output shape:", model.output_shape)  # Should be (None, 3) for 3 classes

class_names = ['Normal', 'Violence', 'Weaponized']
input_shape = model.input_shape[1:]  # Should be (15, 128, 128, 3)

# Create output directory
output_dir = "classified_frames"
os.makedirs(output_dir, exist_ok=True)

def process_frame(frame):
    """Process individual frame to model input specs"""
    frame = cv2.resize(frame, (input_shape[2], input_shape[1]))  # Resize to 128x128
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
    frame = frame / 255.0  # Normalize to [0,1]
    return frame

def classify_video(video_path, sequence_length=15):
    """Main processing function with complete prediction handling"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    frame_buffer = deque(maxlen=sequence_length)
    frame_count = 0
    current_predictions = [0, 0, 0]  # Initialize all class probabilities
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(os.path.join(output_dir, 'output_video.mp4'), fourcc, fps, (width, height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process frame and add to buffer
        processed_frame = process_frame(frame)
        frame_buffer.append(processed_frame)
        
        # When we have enough frames for a sequence
        if len(frame_buffer) == sequence_length:
            sequence = np.expand_dims(np.array(frame_buffer), axis=0)
            
            # Get full prediction array
            predictions = model.predict(sequence, verbose=0)  # Get first (and only) batch item
            print("Full prediction array:", predictions)  # Debug output
            
            # Verify prediction array length matches classes
            if len(predictions) != len(class_names):
                print(f"Warning: Expected {len(class_names)} outputs, got {len(predictions)}")
                current_predictions = predictions  # Store whatever we got
            else:
                current_predictions = predictions
        
        # Prepare display frame with all prediction info
        display_frame = frame.copy()
        
        # Display all class probabilities
        y_offset = 40
        for i, (class_name, prob) in enumerate(zip(class_names, current_predictions)):
            color = (0, 255, 0) if i == 0 else ((0, 165, 255) if i == 1 else (0, 0, 255))
            text = f"{class_name}: {prob:.4f}"
            cv2.putText(display_frame, text, (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX,
                       0.7, color, 2, cv2.LINE_AA)
            y_offset += 30
        
        # Highlight the highest probability class
        dominant_class = np.argmax(current_predictions)
        cv2.rectangle(display_frame, (10, 10), (50, 50), 
                     (0, 255, 0) if dominant_class == 0 else 
                     (0, 165, 255) if dominant_class == 1 else 
                     (0, 0, 255), -1)
        
        # Add frame counter
        cv2.putText(display_frame, f"Frame: {frame_count}", 
                   (20, y_offset + 10), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.7, (200, 200, 0), 2)
        
        # Display
        cv2.imshow('Violence Detection', display_frame)
        
        # Save frame
        output_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(output_path, display_frame)
        
        # Write to output video
        out.write(display_frame)
        
        # Control playback
        frame_count += 1
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Processing complete. Saved {frame_count} frames to {output_dir}")

# Example usage
video_path = "./t_v011_converted.avi"
classify_video(video_path)

Model input shape: (None, 15, 128, 128, 3)
Model output shape: (None, 1)
Full prediction array: [[0.97073334]]


TypeError: unsupported format string passed to numpy.ndarray.__format__

: 

In [1]:


import numpy as np
import argparse
import pickle
import cv2
import os
import time
from keras.models import load_model
from collections import deque

import numpy as np
import pickle
import matplotlib
matplotlib.use('module://ipykernel.pylab.backend_inline')  # Before importing pyplot
import matplotlib.pyplot as plt



IMG_SIZE = 128

def print_results(video, limit=None):
        fig=plt.figure(figsize=(16, 30))
        if not os.path.exists('output'):
            os.mkdir('output')

        print("Loading model ...")
        model = load_model('./violence_detection_model.h5')
        Q = deque(maxlen=128)

        vs = cv2.VideoCapture(video)
        writer = None
        (W, H) = (None, None)
        count = 0     
        while True:
                (grabbed, frame) = vs.read()
                ID = vs.get(1)
                if not grabbed:
                    break
                try:
                    if (ID % 7 == 0):
                        count = count + 1
                        n_frames = len(frame)
                        
                        if W is None or H is None:
                            (H, W) = frame.shape[:2]

                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        output = cv2.resize(frame, (512, 360)).copy()
                        frame = cv2.resize(frame, (128, 128)).astype("float32")
                        frame = frame.reshape(IMG_SIZE, IMG_SIZE, 3) / 255
                        preds = model.predict(np.expand_dims(frame, axis=0))[0]
                        Q.append(preds)

                        results = np.array(Q).mean(axis=0)
                        i = (preds > 0.56)[0] #np.argmax(results)

                        label = i

                        text = "Violence: {}".format(label)
                        #print('prediction:', text)
                        file = open("output.txt",'w')
                        file.write(text)
                        file.close()

                        color = (0, 255, 0)

                        if label:
                            color = (255, 0, 0) 
                        else:
                            color = (0, 255, 0)

                        cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
                                1, color, 3)


                        # saving mp4 with labels but cv2.imshow is not working with this notebook
                        if writer is None:
                                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                                writer = cv2.VideoWriter("output.mp4", fourcc, 60,
                                        (W, H), True)

                        writer.write(output)
                        #cv2.imshow("Output", output)

                        fig.add_subplot(8, 3, count)
                        plt.imshow(output)

                    if limit and count > limit:
                        break

                except:
                    break 
        
        plt.show()
        print("Cleaning up...")
        if writer is not None:
            writer.release()
        vs.release()
        

In [2]:
import numpy as np
import cv2
import os
from keras.models import load_model
from collections import deque

# Constants
SEQ_LENGTH = 15  # Number of frames in each sequence
IMG_SIZE = 128
DISPLAY_SIZE = (512, 360)
VIOLENCE_THRESHOLD = 0.56  # Confidence threshold for violence detection

def process_full_video(video_path):
    # Create output directory
    if not os.path.exists('output'):
        os.makedirs('output')

    print("Loading model...")
    model = load_model('./violence_detection_model.h5')
    
    # Initialize frame buffer
    frame_buffer = deque(maxlen=SEQ_LENGTH)
    predictions = deque(maxlen=SEQ_LENGTH)  # Store recent predictions
    
    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    # Get video properties for writer
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter('output.mp4', fourcc, fps, DISPLAY_SIZE)
    
    # Create display window
    cv2.namedWindow("Violence Detection", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Violence Detection", *DISPLAY_SIZE)
    
    frame_count = 0
    current_status = "Initializing..."
    current_confidence = 0.0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        try:
            # Process frame
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            display_frame = cv2.resize(frame_rgb, DISPLAY_SIZE)
            
            # Prepare frame for model
            model_frame = cv2.resize(frame_rgb, (IMG_SIZE, IMG_SIZE))
            model_frame = model_frame.astype("float32") / 255.0
            frame_buffer.append(model_frame)
            
            # When we have enough frames for a sequence
            if len(frame_buffer) == SEQ_LENGTH:
                # Create sequence tensor (1, 15, 128, 128, 3)
                sequence = np.expand_dims(np.array(frame_buffer), axis=0)
                
                # Make prediction
                pred = model.predict(sequence, verbose=0)[0]
                predictions.append(pred)
                
                # Apply temporal smoothing (average of last SEQ_LENGTH predictions)
                avg_prediction = np.mean(predictions) if predictions else 0.0
                is_violence = avg_prediction > VIOLENCE_THRESHOLD
                current_status = "Violence" if is_violence else "Normal"
                current_confidence = avg_prediction
            
            # Add annotation to display frame (using current status)
            color = (0, 0, 255) if current_status == "Violence" else (0, 255, 0)
            cv2.putText(display_frame, 
                       f"{current_status} ({current_confidence:.2f})", 
                       (35, 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 
                       1, color, 2)
            
            # Save text output (continuously updating)
            with open("output.txt", 'w') as f:
                f.write(f"{current_status} ({current_confidence:.2f})")
            
            # Write and display frame
            writer.write(cv2.cvtColor(display_frame, cv2.COLOR_RGB2BGR))
            cv2.imshow("Violence Detection", cv2.cvtColor(display_frame, cv2.COLOR_RGB2BGR))
            
            frame_count += 1
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
                
        except Exception as e:
            print(f"Error processing frame {frame_count}: {str(e)}")
            break
    
    # Cleanup
    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    print(f"Finished processing {frame_count} frames")

# Example usage
video_path = "./video/w006_converted.avi"
process_full_video(video_path)

Loading model...


Finished processing 171 frames


TKINTER W 2 CLASS NAMES

In [1]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
import numpy as np
import cv2
import os
from keras.models import load_model
from collections import deque
import threading

# Constants
SEQ_LENGTH = 15
IMG_SIZE = 128
DISPLAY_SIZE = (640, 360)
VIOLENCE_THRESHOLD = 0.56

class ViolenceDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Violence Detection System")
        self.root.geometry("800x600")
        
        # Model and video processing variables
        self.model = None
        self.cap = None
        self.running = False
        self.frame_buffer = deque(maxlen=SEQ_LENGTH)
        self.predictions = deque(maxlen=SEQ_LENGTH)
        self.current_status = "Ready"
        self.current_confidence = 0.0
        self.last_prediction = 0.0  # Store the last prediction value
        
        # Create containers
        self.main_frame = tk.Frame(self.root)
        self.greeting_frame = self.create_greeting_frame()
        self.detection_frame = None
        
        # Show greeting frame initially
        self.show_greeting_frame()
        
    def create_greeting_frame(self):
        frame = tk.Frame(self.main_frame)
        
        tk.Label(frame, text="Welcome to Violence Detection System", 
                font=("Helvetica", 18, "bold")).pack(pady=20)
        
        tk.Label(frame, text="This application detects violent behavior in video sequences", 
                font=("Helvetica", 12)).pack(pady=10)
        
        tk.Label(frame, text="Using deep learning to analyze video frames", 
                font=("Helvetica", 12)).pack(pady=10)
        
        start_button = tk.Button(frame, text="Get Started", command=self.start_detection_flow,
                                font=("Helvetica", 14), bg="#4CAF50", fg="white")
        start_button.pack(pady=30)
        
        return frame
    
    def create_detection_frame(self):
        frame = tk.Frame(self.main_frame)
        
        # Video display
        self.video_label = tk.Label(frame)
        self.video_label.pack(pady=10)
        
        # Control panel
        control_frame = tk.Frame(frame)
        control_frame.pack(pady=10)
        
        self.select_button = tk.Button(control_frame, text="Select Video", command=self.select_video)
        self.select_button.pack(side=tk.LEFT, padx=5)
        
        self.start_button = tk.Button(control_frame, text="Start", command=self.start_detection, 
                                     state=tk.DISABLED)
        self.start_button.pack(side=tk.LEFT, padx=5)
        
        self.stop_button = tk.Button(control_frame, text="Stop", command=self.stop_detection,
                                    state=tk.DISABLED)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        # Status display
        self.status_var = tk.StringVar(value="Status: Ready")
        tk.Label(frame, textvariable=self.status_var, font=("Helvetica", 12)).pack(pady=5)
        
        # Confidence display
        self.confidence_var = tk.StringVar(value="")
        tk.Label(frame, textvariable=self.confidence_var, font=("Helvetica", 12)).pack(pady=5)
        
        # Back button
        tk.Button(frame, text="Back to Home", command=self.show_greeting_frame).pack(pady=10)
        
        return frame
    
    def show_greeting_frame(self):
        if self.detection_frame:
            self.detection_frame.pack_forget()
        self.greeting_frame.pack(fill=tk.BOTH, expand=True)
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
    def start_detection_flow(self):
        self.greeting_frame.pack_forget()
        if not self.detection_frame:
            self.detection_frame = self.create_detection_frame()
        self.detection_frame.pack(fill=tk.BOTH, expand=True)
        
        # Load model in background
        threading.Thread(target=self.load_model, daemon=True).start()
    
    def load_model(self):
        self.status_var.set("Status: Loading model...")
        try:
            self.model = load_model('./violence_detection_model.h5')
            self.status_var.set("Status: Model loaded. Select a video.")
            self.select_button.config(state=tk.NORMAL)
        except Exception as e:
            self.status_var.set(f"Status: Error loading model - {str(e)}")
    
    def select_video(self):
        file_path = filedialog.askopenfilename(
            title="Select Video File",
            filetypes=[("Video Files", "*.mp4 *.avi *.mov"), ("All Files", "*.*")]
        )
        
        if file_path:
            self.video_path = file_path
            self.status_var.set(f"Status: Selected {os.path.basename(file_path)}")
            self.start_button.config(state=tk.NORMAL)
    
    def start_detection(self):
        if not hasattr(self, 'video_path'):
            return
            
        self.running = True
        self.start_button.config(state=tk.DISABLED)
        self.stop_button.config(state=tk.NORMAL)
        self.select_button.config(state=tk.DISABLED)
        
        # Reset buffers
        self.frame_buffer = deque(maxlen=SEQ_LENGTH)
        self.predictions = deque(maxlen=SEQ_LENGTH)
        self.current_status = "Processing..."
        self.current_confidence = 0.0
        self.last_prediction = 0.0
        
        # Start video processing in a separate thread
        threading.Thread(target=self.process_video, daemon=True).start()
    
    def stop_detection(self):
        self.running = False
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.select_button.config(state=tk.NORMAL)
        if self.cap:
            self.cap.release()
    
    def process_video(self):
        self.cap = cv2.VideoCapture(self.video_path)
        if not self.cap.isOpened():
            self.status_var.set("Status: Error opening video file")
            return
        
        while self.running and self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                break
            
            try:
                # Process frame
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                display_frame = cv2.resize(frame_rgb, DISPLAY_SIZE)
                
                # Prepare frame for model
                model_frame = cv2.resize(frame_rgb, (IMG_SIZE, IMG_SIZE))
                model_frame = model_frame.astype("float32") / 255.0
                self.frame_buffer.append(model_frame)
                
                # When we have enough frames for a sequence
                if len(self.frame_buffer) == SEQ_LENGTH and self.model:
                    # Create sequence tensor (1, 15, 128, 128, 3)
                    sequence = np.expand_dims(np.array(self.frame_buffer), axis=0)
                    
                    # Make prediction
                    pred = self.model.predict(sequence, verbose=0)[0][0]
                    self.predictions.append(pred)
                    self.last_prediction = pred  # Store the latest prediction
                    
                    # Apply temporal smoothing (average of last SEQ_LENGTH predictions)
                    if len(self.predictions) > 0:
                        avg_prediction = np.mean(self.predictions)
                        is_violence = avg_prediction > VIOLENCE_THRESHOLD
                        self.current_status = "Violence" if is_violence else "Normal"
                        self.current_confidence = avg_prediction
                    else:
                        self.current_status = "Processing..."
                        self.current_confidence = 0.0
                else:
                    # If we don't have enough frames yet, use the last prediction
                    if hasattr(self, 'last_prediction'):
                        is_violence = self.last_prediction > VIOLENCE_THRESHOLD
                        self.current_status = "Violence" if is_violence else "Normal"
                        self.current_confidence = self.last_prediction
                
                # Update status display
                self.status_var.set(f"Status: {self.current_status}")
                self.confidence_var.set(f"Confidence: {self.current_confidence:.2f}")
                
                # Add annotation to display frame
                color = (255, 0, 0) if self.current_status == "Violence" else (0, 255, 0)
                cv2.putText(display_frame, 
                           f"{self.current_status} ({self.current_confidence:.2f})", 
                           (35, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 
                           1, color, 2)
                
                # Convert to PhotoImage and display
                img = Image.fromarray(display_frame)
                imgtk = ImageTk.PhotoImage(image=img)
                
                # Update the display in the main thread
                self.root.after(0, self.update_display, imgtk)
                
                # Control playback speed
                cv2.waitKey(25)
                
            except Exception as e:
                self.status_var.set(f"Status: Error processing frame - {str(e)}")
                break
        
        # Cleanup
        if self.cap:
            self.cap.release()
        self.running = False
        self.root.after(0, self.stop_detection)
    
    def update_display(self, image):
        self.video_label.configure(image=image)
        self.video_label.image = image  # Keep a reference
    
    def on_closing(self):
        self.stop_detection()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = ViolenceDetectionApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()

In [7]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
import numpy as np
import cv2
import os
from keras.models import load_model
from collections import deque
import threading
import time

class ViolenceDetectionUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Violence Detection System")
        self.root.geometry("900x700")
        
        # Constants
        self.SEQ_LENGTH = 15
        self.IMG_SIZE = 64
        self.DISPLAY_SIZE = (640, 360)
        self.VIOLENCE_THRESHOLD = 0.56
        
        # Model and video processing variables
        self.model = None
        self.cap = None
        self.running = False
        self.frame_buffer = deque(maxlen=self.SEQ_LENGTH)
        self.predictions = deque(maxlen=self.SEQ_LENGTH)
        self.current_status = "Ready"
        
        # UI Elements
        self.create_widgets()
        
    def create_widgets(self):
        # Main container
        self.main_frame = tk.Frame(self.root, padx=10, pady=10)
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Video display
        self.video_label = tk.Label(self.main_frame, bg='black')
        self.video_label.pack(fill=tk.BOTH, expand=True)
        
        # Control panel
        control_frame = tk.Frame(self.main_frame)
        control_frame.pack(fill=tk.X, pady=10)
        
        # Buttons
        self.load_btn = tk.Button(control_frame, text="Load Model", command=self.load_model)
        self.load_btn.pack(side=tk.LEFT, padx=5)
        
        self.select_btn = tk.Button(control_frame, text="Select Video", command=self.select_video, state=tk.DISABLED)
        self.select_btn.pack(side=tk.LEFT, padx=5)
        
        self.start_btn = tk.Button(control_frame, text="Start", command=self.start_processing, state=tk.DISABLED)
        self.start_btn.pack(side=tk.LEFT, padx=5)
        
        self.stop_btn = tk.Button(control_frame, text="Stop", command=self.stop_processing, state=tk.DISABLED)
        self.stop_btn.pack(side=tk.LEFT, padx=5)
        
        # Status display
        status_frame = tk.Frame(self.main_frame)
        status_frame.pack(fill=tk.X)
        
        self.status_label = tk.Label(status_frame, text="Status: Ready", font=('Helvetica', 12))
        self.status_label.pack(side=tk.LEFT)
        
        # Console output
        self.console = tk.Text(self.main_frame, height=8, state=tk.DISABLED)
        self.console.pack(fill=tk.X, pady=10)
        
    def log_message(self, message):
        self.console.config(state=tk.NORMAL)
        self.console.insert(tk.END, message + "\n")
        self.console.config(state=tk.DISABLED)
        self.console.see(tk.END)
        
    def load_model(self):
        self.log_message("Loading violence detection model...")
        self.load_btn.config(state=tk.DISABLED)
        
        def load_model_thread():
            try:
                self.model = load_model('./model/LRCN_model__Date_Time_2025_05_06__22_51_48__Loss_0.9145573973655701__Accuracy__0.7631579041481018.h5')
                self.log_message("Model loaded successfully!")
                self.select_btn.config(state=tk.NORMAL)
            except Exception as e:
                self.log_message(f"Error loading model: {str(e)}")
                self.load_btn.config(state=tk.NORMAL)
                
        threading.Thread(target=load_model_thread, daemon=True).start()
        
    def select_video(self):
        file_path = filedialog.askopenfilename(
            title="Select Video File",
            filetypes=[("Video Files", "*.mp4 *.avi *.mov"), ("All Files", "*.*")]
        )
        
        if file_path:
            self.video_path = file_path
            self.log_message(f"Selected video: {os.path.basename(file_path)}")
            self.start_btn.config(state=tk.NORMAL)
            
    def start_processing(self):
        if not hasattr(self, 'video_path'):
            return
            
        self.running = True
        self.start_btn.config(state=tk.DISABLED)
        self.stop_btn.config(state=tk.NORMAL)
        self.select_btn.config(state=tk.DISABLED)
        
        # Reset buffers
        self.frame_buffer = deque(maxlen=self.SEQ_LENGTH)
        self.predictions = deque(maxlen=self.SEQ_LENGTH)
        self.current_status = "Processing..."
        
        # Start processing thread
        threading.Thread(target=self.process_video, daemon=True).start()
        
    def stop_processing(self):
        self.running = False
        self.start_btn.config(state=tk.NORMAL)
        self.stop_btn.config(state=tk.DISABLED)
        self.select_btn.config(state=tk.NORMAL)
        if self.cap:
            self.cap.release()
        self.log_message("Processing stopped by user")
        
    def process_video(self):
        self.cap = cv2.VideoCapture(self.video_path)
        if not self.cap.isOpened():
            self.log_message("Error opening video file")
            return
            
        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_count = 0
        self.log_message(f"Starting processing ({total_frames} frames)...")
        
        while self.running and self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                break
                
            try:
                frame_count += 1
                
                # Process frame
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                display_frame = cv2.resize(frame_rgb, self.DISPLAY_SIZE)
                
                # Prepare frame for model
                model_frame = cv2.resize(frame_rgb, (self.IMG_SIZE, self.IMG_SIZE))
                model_frame = model_frame.astype("float32") / 255.0
                self.frame_buffer.append(model_frame)
                
                # When we have enough frames for a sequence
                if len(self.frame_buffer) == self.SEQ_LENGTH and self.model:
                    # Create sequence tensor
                    sequence = np.expand_dims(np.array(self.frame_buffer), axis=0)
                    
                    # Make prediction
                    pred = self.model.predict(sequence, verbose=0)[0][0]
                    self.predictions.append(pred)
                    
                    # Apply temporal smoothing
                    if self.predictions:
                        avg_prediction = np.mean(self.predictions)
                        self.current_status = "Violence" if avg_prediction > self.VIOLENCE_THRESHOLD else "Normal"
                        
                # Update UI
                self.update_display(display_frame)
                
                # Control processing speed
                time.sleep(0.03)  # ~30fps
                
            except Exception as e:
                self.log_message(f"Error processing frame {frame_count}: {str(e)}")
                break
                
        # Cleanup
        if self.cap:
            self.cap.release()
        self.running = False
        self.log_message(f"Finished processing {frame_count} frames")
        self.root.after(0, self.stop_processing)
        
    def update_display(self, frame):
        # Add status text to frame
        color = (255, 0, 0) if self.current_status == "Violence" else (0, 255, 0)
        cv2.putText(frame, 
                   f"{self.current_status}", 
                   (35, 50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 
                   1, color, 2)
        
        # Convert to PhotoImage
        img = Image.fromarray(frame)
        imgtk = ImageTk.PhotoImage(image=img)
        
        # Update status label
        self.status_label.config(text=f"Status: {self.current_status}")
        
        # Update video display
        self.video_label.imgtk = imgtk
        self.video_label.config(image=imgtk)
        
    def on_closing(self):
        self.stop_processing()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = ViolenceDetectionUI(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()

In [4]:
import numpy as np
import cv2
import os
from keras.models import load_model
from collections import deque

# Constants
SEQ_LENGTH = 1000  # Number of frames in each sequence
IMG_SIZE = 64
DISPLAY_SIZE = (512, 360)
VIOLENCE_THRESHOLD = 0.56  # Confidence threshold for violence detection

def process_full_video(video_path):
    # Create output directory
    if not os.path.exists('output'):
        os.makedirs('output')

    print("Loading model...")
    model = load_model('./model/LRCN_model__Date_Time_2025_05_06__22_51_48__Loss_0.9145573973655701__Accuracy__0.7631579041481018.h5')
    
    # Initialize frame buffer
    frame_buffer = deque(maxlen=SEQ_LENGTH)
    predictions = deque(maxlen=SEQ_LENGTH)  # Store recent predictions
    
    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    # Get video properties for writer
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter('output.mp4', fourcc, fps, DISPLAY_SIZE)
    
    # Create display window
    cv2.namedWindow("Violence Detection", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Violence Detection", *DISPLAY_SIZE)
    
    frame_count = 0
    current_status = "Initializing..."
    current_confidence = 0.0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        try:
            # Process frame
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            display_frame = cv2.resize(frame_rgb, DISPLAY_SIZE)
            
            # Prepare frame for model
            model_frame = cv2.resize(frame_rgb, (IMG_SIZE, IMG_SIZE))
            model_frame = model_frame.astype("float32") / 255.0
            frame_buffer.append(model_frame)
            
            # When we have enough frames for a sequence
            if len(frame_buffer) == SEQ_LENGTH:
                # Create sequence tensor (1, 15, 128, 128, 3)
                sequence = np.expand_dims(np.array(frame_buffer), axis=0)
                
                # Make prediction
                pred = model.predict(sequence, verbose=0)[0]
                predictions.append(pred)
                print(predictions)
                # Apply temporal smoothing (average of last SEQ_LENGTH predictions)
                avg_prediction = np.mean(predictions) if predictions else 0.0
                is_violence = avg_prediction > VIOLENCE_THRESHOLD
                current_status = "Shoplifting" if is_violence else "No Shoplifting"
                current_confidence = avg_prediction
            
            # Add annotation to display frame (using current status)
            color = (0, 0, 255) if current_status == "Violence" else (0, 255, 0)
            cv2.putText(display_frame, 
                       f"{current_status} ({current_confidence:.2f})", 
                       (35, 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 
                       1, color, 2)
            
            # Save text output (continuously updating)
            with open("output.txt", 'w') as f:
                f.write(f"{current_status} ({current_confidence:.2f})")
            
            # Write and display frame
            writer.write(cv2.cvtColor(display_frame, cv2.COLOR_RGB2BGR))
            cv2.imshow("Violence Detection", cv2.cvtColor(display_frame, cv2.COLOR_RGB2BGR))
            
            frame_count += 1
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
                
        except Exception as e:
            print(f"Error processing frame {frame_count}: {str(e)}")
            break
    
    # Cleanup
    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    print(f"Finished processing {frame_count} frames")

# Example usage
video_path = "./video/VIDEO-2025-05-06-17-11-33 (1).mp4"
process_full_video(video_path)

Loading model...


Finished processing 788 frames
